# **Set up**

In [9]:
!pip install -q openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.9/262.9 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.4 MB/s eta 0:00:00


In [30]:
!pip install -q rouge_score

In [10]:
from openai import OpenAI

client = OpenAI(
  api_key='YOUR_API_KEY',
)

In [11]:
def get_gpt_response(prompt, max_tokens=200):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}],
        # max_tokens=500,
        # n=1,
        # stop="\n",  # Set a stop sequence here
        # temperature=0.7,
    )
    return response.choices[0].message.content

# **Prompting**

In [15]:
input_text = '''
Hợp đồng VFF ký với HLV Philippe Troussier có thời hạn hơn ba năm, từ ngày 1-3-2023 đến 31-7-2026.
Trong đó, mục tiêu lớn nhất là giúp đội tuyển Việt Nam phấn đấu có vé dự World Cup 2026.

Trong năm đầu tiên của hợp đồng, nhiệm vụ đầu tiên của HLV Philippe Troussier là giúp đội tuyển U22 Việt Nam
vào đến trận chung kết SEA Games 32 tại Campuchia đã không hoàn thành.

Còn năm 2024, chỉ tiêu cho HLV Philippe Troussier là đưa đội tuyển Việt Nam vượt qua vòng loại thứ 2 World Cup 2026 và giành huy chương ở AFF Cup 2024.
'''

In [16]:
# Standard Prompting
prompt = '''
Tóm tắt đoặn văn sau trong khoảng 2-3 dòng:
{}
'''.format(input_text)
response1 = get_gpt_response(prompt)
print(response1)

Hợp đồng VFF ký với HLV Philippe Troussier kéo dài hơn ba năm, nhấn mạnh vào mục tiêu giúp đội tuyển Việt Nam có vé dự World Cup 2026. Nhiệm vụ đầu tiên của HLV là đưa đội U22 vào chung kết SEA Games 32, nhưng không thành công. Trọng tâm năm 2024 là vượt qua vòng loại thứ 2 World Cup 2026 và chiến thắng ở AFF Cup 2024.


Hợp đồng VFF ký với HLV Philippe Troussier kéo dài hơn ba năm, nhấn mạnh vào mục tiêu giúp đội tuyển Việt Nam có vé dự World Cup 2026. Nhiệm vụ đầu tiên của HLV là đưa đội U22 vào chung kết SEA Games 32, nhưng không thành công. Trọng tâm năm 2024 là vượt qua vòng loại thứ 2 World Cup 2026 và chiến thắng ở AFF Cup 2024.


In [17]:
# Directional Stimulus Prompting
prompt = '''
Tóm tắt đoặn văn sau trong khoảng 2-3 dựa trên các gợi ý.
{}
Gợi ý: "Hợp đồng", "VFF", "HLV Philippe Troussier", "thời hạn", "World Cup 2026", "đội tuyển Việt Nam", "SEA Games 32", "Campuchia", "vòng loại thứ 2", "AFF Cup 2024"
'''.format(input_text)
response2 = get_gpt_response(prompt)
print(response2)

Hợp đồng giữa VFF và HLV Philippe Troussier kéo dài hơn ba năm, từ 1-3-2023 đến 31-7-2026, với mục tiêu chính là giúp đội tuyển Việt Nam giành vé dự World Cup 2026. Năm đầu tiên, HLV Troussier không thể đưa đội U22 Việt Nam vào chung kết SEA Games 32 tại Campuchia. Năm 2024, chỉ tiêu cho ông là đưa đội tuyển vượt qua vòng loại thứ 2 World Cup 2026 và đạt huy chương tại AFF Cup 2024.


Hợp đồng giữa VFF và HLV Philippe Troussier kéo dài hơn ba năm, từ 1-3-2023 đến 31-7-2026, với mục tiêu chính là giúp đội tuyển Việt Nam giành vé dự World Cup 2026. Năm đầu tiên, HLV Troussier không thể đưa đội U22 Việt Nam vào chung kết SEA Games 32 tại Campuchia. Năm 2024, chỉ tiêu cho ông là đưa đội tuyển vượt qua vòng loại thứ 2 World Cup 2026 và đạt huy chương tại AFF Cup 2024.


In [36]:
from rouge_score import rouge_scorer

# Load metric ROUGE
rouge_metric = rouge_scorer.RougeScorer(['rouge1'], use_stemmer=True)


# Tính ROUGE-1
score1 = rouge_metric.score(response1, input_text)["rouge1"]
score2 = rouge_metric.score(response2, input_text)["rouge1"]

print(score1)
print(score2)

Score(precision=0.6066666666666667, recall=0.91, fmeasure=0.7280000000000001)
Score(precision=0.7266666666666667, recall=0.9646017699115044, fmeasure=0.8288973384030419)


Với độ đo ROUGE-1, các giá trị precision, recall và f-measure càng cao thì sự tương đồng giữa các câu càng cao.

Do đó response2 khi có các gợi ý có **độ tương đồng** với câu cần summarize **cao hơn** so với response1

**Ngoài ra, có thể sử dụng 1 LLM nhỏ hơn và có thể tune được (ví dụ như mô hình T5) để tạo sinh các gợi ý/hướng dẫn**